In [14]:
# Standard libraries
from cellpose import models, core, plot, utils, denoise
from glob import glob

from skimage import exposure, measure

import numpy as np
import matplotlib.pyplot as plt
import cv2

# Custom packages
from auxiliary.data.dataset_ht import HtDataset
from auxiliary.data.imaging import read_nii
from auxiliary.data import imaging  

# Configurations
use_gpu = core.use_gpu()
print(f"GPU activated: {use_gpu}")

from cellpose.io import logger_setup
logger_setup();

2024-12-03 12:11:53,924 [INFO] ** TORCH CUDA version installed and working. **
GPU activated: True
2024-12-03 12:11:53,926 [INFO] WRITING LOG OUTPUT TO /home/imarcoss/.cellpose/run.log
2024-12-03 12:11:53,926 [INFO] 
cellpose version: 	3.0.10 
platform:       	linux 
python version: 	3.11.0 
torch version:  	2.4.0+cu121


In [11]:
# paht = glob(f'/run/user/1003/gvfs/smb-share:server=tierra.cnic.es,share=sc/LAB_MT/LAB/Oscar/PROJECT_transition_zone/Paper/Data/Colum_FGFinh_MeisDKO/MeisdKO_WT_BCat_Columnarity/test_plantseg/cropped/Normalized_images/denoise/*.tif')

paht = [f'/run/user/1003/gvfs/smb-share:server=tierra.cnic.es,share=sc/LAB_MT/LAB/Oscar/PROJECT_transition_zone/Paper/Data/Colum_FGFinh_MeisDKO/MeisdKO_WT_BCat_Columnarity/test_plantseg/cropped_denoised/Normalized/C4-MeisDKO_Par3488_PhallRhod_Bcat647_17_11_23.lif - Series001_cropped.tif']


imgs = list(map(imaging.read_image, paht))

In [12]:
[print(np.array(img).shape) for img in imgs]

(1614, 954, 94)


[None]

In [13]:
denoised = []
for i, img in enumerate(imgs):    
    model = denoise.DenoiseModel(
        gpu=use_gpu,
        model_type="denoise_cyto3",
    )
    
    img = np.array(img)
    den = np.swapaxes(np.swapaxes([
        np.array(model.eval(
            img[..., z], channels=[0, 0], diameter=40, z_axis=2
        )) for z in range(img.shape[-1])
    ], 0, 2), 0, 1)
    
    if den.ndim == 4:
        den = den[..., 0]
    
    imaging.save_prediction(den, paht[i].replace('.tif', '_denoised.tif'), 'XYZ', verbose=1)

2024-12-03 12:04:42,113 [INFO] >> denoise_cyto3 << model set to be used
2024-12-03 12:04:42,133 [INFO] ** TORCH CUDA version installed and working. **
2024-12-03 12:04:42,134 [INFO] >>>> using GPU
2024-12-03 12:04:42,309 [INFO] >>>> model diam_mean =  30.000 (ROIs rescaled to this size during training)

Saving prediction: /run/user/1003/gvfs/smb-share:server=tierra.cnic.es,share=sc/LAB_MT/LAB/Oscar/PROJECT_transition_zone/Paper/Data/Colum_FGFinh_MeisDKO/MeisdKO_WT_BCat_Columnarity/test_plantseg/cropped_denoised/Normalized/C4-MeisDKO_Par3488_PhallRhod_Bcat647_17_11_23.lif - Series001_cropped_denoised.tif


In [17]:
paht = glob(f'/run/user/1003/gvfs/smb-share:server=tierra.cnic.es,share=sc/LAB_MT/LAB/Oscar/PROJECT_transition_zone/Paper/Data/Colum_FGFinh_MeisDKO/MeisdKO_WT_BCat_Columnarity/test_plantseg/GASP/*.tiff')


imgs = list(map(imaging.read_image, paht))
[print(np.array(img).shape) for img in imgs]

(1614, 954, 94)
(1614, 954, 94)
(1614, 954, 94)
(1614, 954, 94)
(1614, 954, 94)
(1614, 954, 94)


[None, None, None, None, None, None]

In [19]:
for i, img in enumerate(imgs):
    img = np.array(img)
    labeled = measure.label(img)
    sizes = np.bincount(labeled.ravel())
    
    # Exclude bck
    sizes[0] = 0
    
    large_labels = np.where(sizes > 21000)[0]
    large_mask = np.isin(labeled, large_labels)
    
    img[large_mask] = 0
    
    aux = paht[i].split('/')
    name = aux[-1].replace('.tif', '_filtered.tif')
    path = '/'.join(aux[:-1])
    
    imaging.save_prediction(
        img, f'{path}/Filtered/{name}', 
        'XYZ', verbose=1
    )


Saving prediction: /run/user/1003/gvfs/smb-share:server=tierra.cnic.es,share=sc/LAB_MT/LAB/Oscar/PROJECT_transition_zone/Paper/Data/Colum_FGFinh_MeisDKO/MeisdKO_WT_BCat_Columnarity/test_plantseg/GASP/Filtered/C2-MeisDKO_denoised_filtered.tiff

Saving prediction: /run/user/1003/gvfs/smb-share:server=tierra.cnic.es,share=sc/LAB_MT/LAB/Oscar/PROJECT_transition_zone/Paper/Data/Colum_FGFinh_MeisDKO/MeisdKO_WT_BCat_Columnarity/test_plantseg/GASP/Filtered/C2-EZ_denoised_filtered.tiff

Saving prediction: /run/user/1003/gvfs/smb-share:server=tierra.cnic.es,share=sc/LAB_MT/LAB/Oscar/PROJECT_transition_zone/Paper/Data/Colum_FGFinh_MeisDKO/MeisdKO_WT_BCat_Columnarity/test_plantseg/GASP/Filtered/C4-EZ_denoised_filtered.tiff

Saving prediction: /run/user/1003/gvfs/smb-share:server=tierra.cnic.es,share=sc/LAB_MT/LAB/Oscar/PROJECT_transition_zone/Paper/Data/Colum_FGFinh_MeisDKO/MeisdKO_WT_BCat_Columnarity/test_plantseg/GASP/Filtered/C4-MeisDKO_denoised_filtered.tiff

Saving prediction: /run/user/1003